In [1]:
import pandas as pd
import numpy as np

In [2]:
# import urllib library
from urllib.request import urlopen
  
# import json
import json
# store the URL in url as 
# parameter for urlopen
url = "https://classes.cornell.edu/api/2.0/search/classes.json?roster=SP22&subject=CS" # get SP 22 CS classes
  
# store the response of URL
response = urlopen(url)
  
# storing the JSON response 
# from url in data
data_json = json.loads(response.read())


In [3]:
classes = data_json['data']['classes']

In [4]:
# CS 1110, Bracy, start time, end time, location, description, prereqs/coreqs, semester offered

for subject in ["CS", "INFO", "ECE"]: #, "ECON"]"ENGL", "PHIL"]:
    url = f"https://classes.cornell.edu/api/2.0/search/classes.json?roster=SP22&subject={subject}"
    response = urlopen(url)
  
    # storing the JSON response 
    # from url in data
    data_json = json.loads(response.read())
    
    classes = data_json['data']['classes']
    for class_ in classes:
        if (int(class_['catalogNbr'])) > 5900:
            continue
        #print(class_['subject'] + class_['catalogNbr'] + class_['titleLong'])
        #print(class_)
        obj = class_['enrollGroups'][0]['classSections'][0]['meetings'][0]['instructors'][0]
        #print(obj)
        print(obj['firstName'] + ' ' + obj['middleName'] + ' ' + obj['lastName'])

Anne  Bracy
Kit-Yee Daisy  Fan
Kit-Yee Daisy  Fan
Craig C. Frey
Royal  Westwater
Jon M. Kleinberg
Sumanta  Basu
Shaun B. Nichols
Walker Mcmillan White
Ronald  DiNapoli
Michael Ryan Clarkson
Michael Ryan Clarkson
Kyle J. Harms
Anke  van Zuylen
Nate  Foster
Walker Mcmillan White
Anne  Bracy
Nils  Napp
Dexter  Kozen
Andrew C. Myers
Andrew C. Myers
Walker Mcmillan White
Michael Ryan Clarkson
David S. Bindel
Robbert  VanRenesse
Robbert  VanRenesse
Bharath  Hariharan
Kevin Michael Ellis
Kevin Michael Ellis
Marten  van Schijndel
Hadas  Kress-Gazit
Christopher Matthew De Sa
Karthik  Sridharan
Sarah A Dean
Dexter  Kozen
Eshan  Chattopadhyay
Noah  Stephens-Davidowitz
Robert David Kleinberg
Cristobal  Cheyre Forestier
Lillian  Lee
Kavita  Bala
Wen  Sun
Andrew C. Myers
Andrew C. Myers
Andrea  Lodi
Curran Daniel Muhlberger
Walker Mcmillan White
Michael Ryan Clarkson
David S. Bindel
Rajalakshmi  Nandakumar
Danny  Aslam-Perez
Robbert  VanRenesse
Robbert  VanRenesse
Ken  Birman
Ari  Juels
Helen  Nisse

In [5]:
# CS 1110, Bracy, start time, end time, location, description, prereqs/coreqs, semester offered
df = []

for subject in ["CS", "INFO", "ECE"]: #, "ECON"]"ENGL", "PHIL"]:
    url = f"https://classes.cornell.edu/api/2.0/search/classes.json?roster=SP22&subject={subject}"
    response = urlopen(url)
  
    # storing the JSON response 
    # from url in data
    data_json = json.loads(response.read())
    
    classes = data_json['data']['classes']

    for class_ in classes:
        #print(class_)
        if (int(class_['catalogNbr'])) > 5900:
            continue
        #print(class_)
        obj = class_['enrollGroups'][0]['classSections'][0]['meetings'][0]

        obj2 = obj['instructors'][0]
        #print(obj)

        startTime = obj['timeStart']
        endTime = obj['timeEnd']
        location = obj['facilityDescr']
        x = [class_['subject'] , class_['catalogNbr'] , class_['titleLong'], obj2['firstName'] + ' ' + obj2['middleName'] + ' ' + obj2['lastName'], startTime, endTime, location, class_['description'], class_['catalogPrereqCoreq'], class_['catalogWhenOffered']]
        df.append(x)

In [6]:
df1 = pd.DataFrame(np.array(df))
df1

,0,1,2,3,4,5,6,7,8,9
0,CS,1110,Introduction to Computing Using Python,Anne Bracy,09:05AM,09:55AM,Bailey Hall 101,Programming and problem solving using Python. ...,,"Fall, Spring, Summer."
1,CS,1112,Introduction to Computing Using MATLAB,Kit-Yee Daisy Fan,02:40PM,03:30PM,Hollister Hall B14,Programming and problem solving using MATLAB. ...,"Corequisite: MATH 1110, MATH 1910, or equivalent.","Fall, Spring."
2,CS,1132,Short Course in MATLAB,Kit-Yee Daisy Fan,02:40PM,03:30PM,Phillips Hall 219,Introduction to the MATLAB programming languag...,,"Fall, Spring."
3,CS,1133,Short Course in Python,Craig C. Frey,12:25PM,01:15PM,Upson Hall 142,Introduction to the Python programming languag...,,"Fall, Spring."
4,CS,1300,Introductory Design and Programming for the Web,Royal Westwater,11:25AM,12:40PM,Olin Hall 165,The World Wide Web is both a technology and a ...,,"Fall, Spring."
...,...,...,...,...,...,...,...,...,...,...
165,ECE,5772,Autonomous Mobile Robots,Hadas Kress-Gazit,09:40AM,10:55AM,Hollister Hall 110,Creating robots capable of performing complex ...,,Spring.
166,ECE,5780,Computer Analysis of Biomed Images,Anthony P. Reeves,09:40AM,10:55AM,Phillips Hall 307,Powerful imaging modalities with attending com...,Prerequisite: ECE 5470 or equivalent.,Spring.
167,ECE,5790,Advanced High-Speed and RF Integrated Circuits,Alyosha Christopher Molnar,11:25AM,12:40PM,Phillips Hall 403,Principles of analog integrated circuit design...,Prerequisite: ECE 4530.,Spring.
168,ECE,5840,Introduction to Controlled Fusion: Principles ...,David A. Hammer,11:20AM,12:10PM,Phillips Hall 407,Introduction to the physical principles and va...,"Prerequisite: one of the following: PHYS 1112,...",Spring.
